## Create .yaml file to get the missing data

In [1]:
import os
import numpy as np
import time
import pandas as pd
from pathlib import Path

BASE_PATH = "C:\\Users\\MHJGuillermo\\PycharmProjects\\Doctorado\\Ising\\isingchat\\venv"
DATA_PATH = Path(os.path.join(BASE_PATH,"data\\regular\\infinite-size\\")).resolve()

### Finding the critical temperature using extrapol

In [2]:
def serie_divergente(n: int,p: float, z: float=1, a: float=1):
    res = 0
    for i in np.arange(0,n,1):
        res = res + z**i/((i+a)**p)
    return res


def model_function(x: float,p,coefs):
    """J/k_BT model, x = nv"""
    a1,a2 = coefs
    return a1*(1/serie_divergente(x,p,z=1,a=a2))

In [4]:
## get points for finite
BASE_DATA_PATH = os.path.join(BASE_PATH,DATA_PATH)
# get points of Cv_mx
file_all_tc = 'regular_all_inv_Tc.csv'
df_inv_all_tc_finite = pd.read_csv(os.path.join(BASE_DATA_PATH,file_all_tc))
# get params of model
file_params = "inv_Tc_vs_inv_nv_extrapol_hurwitz_2_parms_with_nv_5plus_and_error_params_curves.csv"
df_params_finite = pd.read_csv(os.path.join(BASE_DATA_PATH,"Extrapolation",file_params),index_col=0)
# get extrapol val
file_extrapol_val = "inv_Tc_vs_inv_nv_extrapol_hurwitz_2_parms_with_nv_5plus_and_error_extrapol_values.csv"
df_extrapol_val_finite = pd.read_csv(os.path.join(BASE_DATA_PATH,"Extrapolation",file_extrapol_val),index_col=0)

In [11]:
list_exp_p = [1.0,1.05,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9]
list_nv = [26,27,28,29,30]
for exp_p in list_exp_p:
    path_file = os.path.join(BASE_DATA_PATH,"centrosym","p_{}".format(exp_p))
    try:
        os.mkdir(path_file)
    except FileExistsError:
        print('The directory {} has been created before'.format(path_file))
    for nv in list_nv:
        column = 'P'+str(exp_p)
        # get coefs adjusted
        coefs = df_params_finite[column].values[:2]
        print('coefs: {}'.format(coefs))
        extrapol_temp = 1/model_function(nv,exp_p,coefs)
        print('extrapol_temp: {}'.format(extrapol_temp))
        parallel = False
        if nv == 26 or nv == 27 or nv == 28:
            min_temp = extrapol_temp - 0.2
            max_temp = extrapol_temp + 0.2
            num_samples = 32
            parallel = True
        elif nv == 29:
            min_temp = extrapol_temp - 0.2
            max_temp = extrapol_temp + 0.2
            num_samples = 16
        else:
            min_temp = extrapol_temp - 0.2
            max_temp = extrapol_temp + 0.2
            num_samples = 8

        listJ=1/np.arange(1,nv+1,1)**exp_p
        file_name = "p-{}_nv-{}_h-0.yml".format(exp_p,nv)
        print("saving file {}".format(os.path.join(path_file,file_name)))
        f = open(os.path.join(path_file,file_name),"w+")
        f.write("# Arbitrary metadata\n")
        f.write("metadata:\n  description: Ising chain with exp_p={} and {} nearest neighbors\n".format(exp_p,nv))
        f.write("\nsystem:\n  params:\n    interactions: {}\n".format(listJ.tolist()))
        f.write("\n    temperature:\n      min: {}\n      max: {}\n      num_samples: {}\n".format(min_temp,max_temp,num_samples))
        f.write("\n    magnetic_field: 0\n")
        f.write("\nexec:\n  parallel: {}".format(parallel))
        f.write("\n\nuse_centrosymmetric: True")
        f.close()
print('Sucess generate all files')

The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\isingchat\venv\data\regular\infinite-size\centrosym\p_1.0 has been created before
coefs: [0.47927304 2.44401088]
extrapol_temp: 5.539496289932271
saving file C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\isingchat\venv\data\regular\infinite-size\centrosym\p_1.0\p-1.0_nv-26_h-0.yml
coefs: [0.47927304 2.44401088]
extrapol_temp: 5.61285068962613
saving file C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\isingchat\venv\data\regular\infinite-size\centrosym\p_1.0\p-1.0_nv-27_h-0.yml
coefs: [0.47927304 2.44401088]
extrapol_temp: 5.683713771136879
saving file C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\isingchat\venv\data\regular\infinite-size\centrosym\p_1.0\p-1.0_nv-28_h-0.yml
coefs: [0.47927304 2.44401088]
extrapol_temp: 5.752249200033604
saving file C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\isingchat\venv\data\regular\infinite-size\centrosym\p_1.0\p-1.0_nv-29_h-0.yml
coefs: [0.47927304 

In [23]:
print( not os.path.exists(os.path.join(path_file,file_name)))

True


In [7]:
listJ = 1/np.arange(1,20+1,1)**1.1
listJ
print(listJ.tolist())
#print("[{}]".format(','.join([num for num in listJ])))

[1.0, 0.4665164957684037, 0.29865281994692067, 0.21763764082403103, 0.1702679845041569, 0.13932646701298948, 0.11759589342757772, 0.10153154954452942, 0.08919350686224783, 0.07943282347242814, 0.07152667656334291, 0.06499809515869194, 0.059519916389315516, 0.05486042411858822, 0.05085101371884506, 0.04736614270344992, 0.044310452642649344, 0.04161024226667093, 0.03920767043037826, 0.03705672245534738]


In [8]:
print([num for num in listJ])

[1.0, 0.4665164957684037, 0.29865281994692067, 0.21763764082403103, 0.1702679845041569, 0.13932646701298948, 0.11759589342757772, 0.10153154954452942, 0.08919350686224783, 0.07943282347242814, 0.07152667656334291, 0.06499809515869194, 0.059519916389315516, 0.05486042411858822, 0.05085101371884506, 0.04736614270344992, 0.044310452642649344, 0.04161024226667093, 0.03920767043037826, 0.03705672245534738]
